In [1]:
!df -h /content

Filesystem      Size  Used Avail Use% Mounted on
overlay         236G   46G  191G  20% /


In [2]:
!cp "/content/drive/MyDrive/Deep Learning/super-resolution-in-video-games/subset.zip" /content/

In [3]:
!unzip -q /content/subset.zip -d /content/

In [4]:
import os
import shutil

src_srcnn = "/content/drive/MyDrive/Deep Learning/SRCNN/results"
dst_srcnn = "/content/results/srcnn"

src_hat = "/content/drive/MyDrive/Deep Learning/HAT/results/HAT-S_SRx4/visualization"
dst_hat = "/content/results/hat"

os.makedirs(dst_srcnn, exist_ok=True)
os.makedirs(dst_hat, exist_ok=True)

shutil.copytree(src_srcnn, dst_srcnn, dirs_exist_ok=True)
shutil.copytree(src_hat, dst_hat, dirs_exist_ok=True)

print("Setup de arquivos finalizado!")

Setup de arquivos finalizado!


In [5]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 726.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [6]:
import os
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import lpips
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr_metric
from skimage.metrics import structural_similarity as ssim_metric
import matplotlib.pyplot as plt

In [17]:
low_res_path = Path("/content/subset/test/lr")
high_res_path = Path("/content/subset/test/hr")
HAT_results_path = Path("/content/results/hat/custom")
SRCNN_vanilla_path = Path("/content/results/srcnn/SRCNN")
SRCNN_sharpened_path = Path("/content/results/srcnn/SRCNN_sharp")

In [8]:
to_tensor = T.ToTensor()

class SRDataset(Dataset):
    def __init__(self, sr_dir, hr_dir, transform=None):
        self.sr_dir = sr_dir
        self.hr_dir = hr_dir
        self.filenames = sorted(os.listdir(sr_dir))
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        name = self.filenames[idx]
        sr_path = os.path.join(self.sr_dir, name)
        hr_path = os.path.join(self.hr_dir, name)

        sr = Image.open(sr_path).convert("RGB")
        hr = Image.open(hr_path).convert("RGB")

        if self.transform:
            sr = self.transform(sr)
            hr = self.transform(hr)

        return {"name": name, "sr": sr, "hr": hr}

class HATDataset(Dataset):
    def __init__(self, sr_dir, hr_dir, suffix="_HAT-S_SRx4", transform=None):
        self.sr_dir = sr_dir
        self.hr_dir = hr_dir
        self.suffix = suffix
        self.filenames = sorted([
            f for f in os.listdir(sr_dir)
            if os.path.exists(os.path.join(hr_dir, f.replace(suffix, "")))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        sr_name = self.filenames[idx]
        base_name = sr_name.replace(self.suffix, "")
        sr_path = os.path.join(self.sr_dir, sr_name)
        hr_path = os.path.join(self.hr_dir, base_name)

        sr = Image.open(sr_path).convert("RGB")
        hr = Image.open(hr_path).convert("RGB")

        if self.transform:
            sr = self.transform(sr)
            hr = self.transform(hr)

        return {"name": base_name, "sr": sr, "hr": hr}

In [25]:
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lpips_fn = lpips.LPIPS(net="alex").to(device)

def evaluate_model(dataset, model_name):
    results = []
    for sample in tqdm(dataset):
        name = sample["name"]
        sr = sample["sr"].unsqueeze(0).to(device)
        hr = sample["hr"].unsqueeze(0).to(device)

        lpips_score = lpips_fn(sr, hr).item()
        sr_np = sr.squeeze(0).permute(1, 2, 0).cpu().numpy()
        hr_np = hr.squeeze(0).permute(1, 2, 0).cpu().numpy()

        psnr_score = psnr_metric(hr_np, sr_np, data_range=1.0)
        ssim_score = ssim_metric(hr_np, sr_np, channel_axis=-1, data_range=1.0)

        results.append({
            "name": name,
            "model": model_name,
            "psnr": psnr_score,
            "ssim": ssim_score,
            "lpips": lpips_score
        })
    return results

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


In [22]:
print(device)

cuda


In [27]:
hat_ds = HATDataset(HAT_results_path, high_res_path, transform=to_tensor)
srcnn_ds = SRDataset(SRCNN_vanilla_path, high_res_path, transform=to_tensor)
sharp_ds = SRDataset(SRCNN_sharpened_path, high_res_path, transform=to_tensor)

from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=3) as executor:
    future_hat = executor.submit(evaluate_model, hat_ds, "HAT")
    future_srcnn = executor.submit(evaluate_model, srcnn_ds, "SRCNN")
    future_sharp = executor.submit(evaluate_model, sharp_ds, "SRCNN+Sharpen")

    print("Aguardando resultados...")

    hat_results = future_hat.result()
    srcnn_results = future_srcnn.result()
    sharp_results = future_sharp.result()

Aguardando resultados...


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [38]:
import pandas as pd
all_results = hat_results + srcnn_results + sharp_results
df_results = pd.DataFrame(all_results)

df_results.groupby("model")[["psnr", "ssim", "lpips"]].mean()

,psnr,ssim,lpips
model,,,
HAT,24.743059,0.754264,0.274224
SRCNN,24.972874,0.746861,0.311735
SRCNN+Sharpen,24.476483,0.741079,0.283635


In [41]:
df_results.to_csv("/content/drive/MyDrive/Deep Learning/results_final.csv")

In [51]:
def plot_image_comparison(name):
    files = {
        "Low-Res": low_res_path / name,
        "Bicubic": None,  # será gerado
        "SRCNN": SRCNN_vanilla_path / name,
        "SRCNN+Sharpen": SRCNN_sharpened_path / name,
        "HAT-S_SRx4": HAT_results_path / f"{name.replace('.png', '')}_HAT-S_SRx4.png",
        "High-Res": high_res_path / name
    }

    images = {}
    for label, path in files.items():
        if label == "Bicubic":
            lr = Image.open(files["Low-Res"]).convert("RGB")
            hr = Image.open(files["High-Res"]).convert("RGB")
            bicubic = lr.resize(hr.size, Image.BICUBIC)
            images[label] = bicubic
        else:
            images[label] = Image.open(path).convert("RGB")

    fig, axs = plt.subplots(1, len(images), figsize=(30, 5))
    for ax, (label, img) in zip(axs, images.items()):
        ax.imshow(img)
        ax.set_title(label)
        ax.axis("off")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Deep Learning/{name}", dpi=450, bbox_inches='tight')
    plt.show()

for i in range(5):
    plot_image_comparison(df_results['name'].iloc[i])

Output hidden; open in https://colab.research.google.com to view.